In [1]:
import logging
import os

# set the directory to the location of the script
try:
    os.chdir("../..")
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


In [2]:
from src.speech.text_to_speech import SpeechSynthesizer
from src.speech.speech_recognizer import StreamingSpeechRecognizerFromBytes
from openai import AzureOpenAI

if "az_speech_recognizer_stream_client" not in locals():
    az_speech_recognizer_stream_client = StreamingSpeechRecognizerFromBytes(
        vad_silence_timeout_ms=800,
        use_semantic_segmentation=False,
        audio_format="pcm",
        candidate_languages=["en-US", "fr-FR", "de-DE", "es-ES", "it-IT"],
        enable_diarisation=True,
        speaker_count_hint=2,
        enable_neural_fe=False,
    )

if "az_speach_synthesizer_client" not in locals():
    az_speach_synthesizer_client = SpeechSynthesizer()

# Ensure Azure OpenAI client is initialized only if not already defined
if "client" not in locals():
    client = AzureOpenAI(
        api_version="2025-02-01-preview",
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    )

[2025-08-07 00:01:54,894] INFO - micro: Azure Monitor tracing initialized for speech recognizer
INFO:micro:Azure Monitor tracing initialized for speech recognizer
[2025-08-07 00:01:54,908] INFO - micro: Creating SpeechConfig with API key authentication
INFO:micro:Creating SpeechConfig with API key authentication
[2025-08-07 00:01:54,924] INFO - micro: Azure Monitor tracing initialized for speech synthesizer
INFO:micro:Azure Monitor tracing initialized for speech synthesizer
[2025-08-07 00:01:54,938] INFO - micro: Creating SpeechConfig with API key authentication
INFO:micro:Creating SpeechConfig with API key authentication
[2025-08-07 00:01:54,951] INFO - micro: Speech synthesizer initialized successfully
INFO:micro:Speech synthesizer initialized successfully


In [3]:
# Define end-of-sentence markers for TTS
TTS_ENDS = {".", "!", "?", ";", "\n"}

# Flags and buffers
is_synthesizing = False
user_buffer = ""
assistant_buffer = ""
tts_thread = None

In [4]:
import os, time, threading


def on_final(text, lang):
    global user_buffer
    user_buffer += text.strip() + "\n"
    print(f"🧾 User (final) in {lang}: {text}")


def assistant_speak(text):
    global is_synthesizing
    print("Hi there, I am a assistant_speak callback!")
    is_synthesizing = True
    print("Syntetixing:", is_synthesizing)
    az_speach_synthesizer_client.start_speaking_text(text)


def on_partial(text, lang):
    global is_synthesizing
    if is_synthesizing:
        # az_speach_synthesizer_client.stop_speaking()
        is_synthesizing = False
    print(f"🗣️ User (partial) in {lang}: {text}")


az_speech_recognizer_stream_client.set_partial_result_callback(on_partial)
az_speech_recognizer_stream_client.set_final_result_callback(on_final)

# Start recognition
az_speech_recognizer_stream_client.start()
print("🎙️ Speak now...")

# Start mic streaming thread
import pyaudio

RATE, CHANNELS, CHUNK = 16000, 1, 1024
audio = pyaudio.PyAudio()
stream = audio.open(
    format=pyaudio.paInt16,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK,
)


def mic_loop():
    while True:
        data = stream.read(CHUNK, exception_on_overflow=False)
        az_speech_recognizer_stream_client.write_bytes(data)


threading.Thread(target=mic_loop, daemon=True).start()

messages = [{"role": "system", "content": "You are a helpful assistant."}]

user_buffer = ""  # This should be filled in by your STT callback as before

while True:
    if user_buffer:
        full_conversation = (
            "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
            + f"\nUser: {user_buffer}"
        )
        messages.append({"role": "user", "content": user_buffer})
        user_buffer = ""  # clear after using

        completion = client.chat.completions.create(
            stream=True,
            messages=messages,
            max_tokens=4096,
            temperature=1.0,
            top_p=1.0,
            model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
        )

        collected_messages = []
        last_tts_request = None

        for chunk in completion:
            if chunk.choices and hasattr(chunk.choices[0].delta, "content"):
                chunk_text = chunk.choices[0].delta.content
                if chunk_text:
                    collected_messages.append(chunk_text)
                    print(chunk_text, end="", flush=True)
                    # Check for sentence end to stream to TTS
                    if chunk_text in TTS_ENDS:
                        text = "".join(collected_messages).strip()
                        last_tts_request = assistant_speak(text)
                        collected_messages.clear()
        print()  # finish line after streaming LLM response

    time.sleep(0.1)

[2025-08-07 00:01:55,267] INFO - micro: Starting recognition from byte stream…
INFO:micro:Starting recognition from byte stream…
[2025-08-07 00:01:55,283] INFO - micro: Speech-SDK prepare_start – format=pcm  neuralFE=False  diar=True
INFO:micro:Speech-SDK prepare_start – format=pcm  neuralFE=False  diar=True
[2025-08-07 00:01:55,300] INFO - micro: Speech-SDK ready (neuralFE=False, diarisation=True, speakers=2)
INFO:micro:Speech-SDK ready (neuralFE=False, diarisation=True, speakers=2)
[2025-08-07 00:01:55,319] INFO - micro: Recognition started.
INFO:micro:Recognition started.


🎙️ Speak now...
🗣️ User (partial) in en-US: hello
🗣️ User (partial) in en-US: hello how are
🗣️ User (partial) in en-US: hello how are you
🧾 User (final) in en-US: Hello, how are you?
Hello!Hi there, I am a assistant_speak callback!
Syntetixing: True


[2025-08-07 00:02:11,066] INFO - micro: [🔊] Starting streaming speech synthesis for text: Hello!...
INFO:micro:[🔊] Starting streaming speech synthesis for text: Hello!...


 I'm just a computer program, so I don't have feelings, but I'm here and ready to help you.Hi there, I am a assistant_speak callback!
Syntetixing: True


[2025-08-07 00:02:11,131] INFO - micro: [🔊] Starting streaming speech synthesis for text: I'm just a computer program, so I don't have feeli...
INFO:micro:[🔊] Starting streaming speech synthesis for text: I'm just a computer program, so I don't have feeli...


...

[2025-08-07 00:02:11,165] INFO - micro: [🔊] Starting streaming speech synthesis for text: How can I assist you today?...
INFO:micro:[🔊] Starting streaming speech synthesis for text: How can I assist you today?...



🗣️ User (partial) in en-US: hello
🗣️ User (partial) in en-US: hello i'm just
🗣️ User (partial) in en-US: hello i'm just a
🗣️ User (partial) in en-US: hello i'm just a computer program
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but i'm
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but i'm here and ready
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but i'm here and ready to
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but i'm here and ready to help
🗣️ User (partial) in en-US: hello i'm just a compu

KeyboardInterrupt: 

🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but i'm here and ready to help you how can i


🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but i'm here and ready to help you how can i assist
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but i'm here and ready to help you how can i assist you
🗣️ User (partial) in en-US: hello i'm just a computer program so i don't have feelings but i'm here and ready to help you how can i assist you today
🧾 User (final) in en-US: Hello, I'm just a computer program so I don't have feelings but I'm here and ready to help you. How can I assist you today?


In [ ]:
az_speech_recognizer_stream_client.close_stream()
az_speach_synthesizer_client.stop_speaking()

[2025-08-07 00:02:24,040] INFO - micro: [🛑] Stopping speech synthesis...
INFO:micro:[🛑] Stopping speech synthesis...


[2025-08-07 00:02:24,178] WARNING - micro: Recognition canceled: SpeechRecognitionCanceledEventArgs(session_id=d01476a8f2404869b08f70ab43d05912, result=SpeechRecognitionResult(result_id=04c69c2191234b0fb145603c54632ccc, text="", reason=ResultReason.Canceled))
[2025-08-07 00:02:24,186] WARNING - micro: Reason: CancellationReason.EndOfStream, Error: 
[2025-08-07 00:02:24,193] INFO - micro: Session stopped.
INFO:micro:Session stopped.
